# Explosive trajectory generator and analyis tools

This notebook allows the generation of a set of trajectories to be used with NuPPN.
The parameters for these trajectories are defined by the mass and metallicity, but also by specific isotope.

In [ ]:
%matplotlib nbagg
from nugridpy import nugridse as mp
from nugridpy import mesa as ms
from nugridpy import ascii_table as at
col = ['m','g','k','b','r','c','orange','pink','0.75','yellow','0.5','#11ff11',\
        '#a34f11','maroon','#14f21b','#122f15','#16fb1a','#aabb11','#abcdef']

# Define a path to the NuGrid VOS, on new wendi this is located here: /data/nugrid_XXX
# XXX can be one of three vos connections: vos, cedar, vos. Somethimes one goes down, so choose a working directory.
data_dir="/data/nugrid_cedar/"
# data_dir="/data/nugrid_rpod4/"
# data_dir="/data/nugrid_vos/"

ms.set_nugrid_path(data_dir)
mp.set_nugrid_path(data_dir)

# Define your mass and metallicity
mass = 12.0
z    = 0.02

# For temp and density peak cells define a range of masses to examine
# These data vales are referred to as mass list from here on
mass_list=[15.0,20.0,12.0,25.0]

#Generate a dictionary of prefixes for the desired mass list
pt_dict_exp_names={x:'pt_exp_{}'.format(int(x)) for x in mass_list}
pt_dict_pre_names={x:'pt_pre_{}'.format(int(x)) for x in mass_list}

#############################################
# Functions used within:

def limiter(I_array):
    j=0
    for i in I_array:
        if i> (5*I_array[0]):
            cut = j
            print(j)
            break
        j +=1
    I_array_output    = I_array[cut:]
    return (I_array_output,cut)

def shell_data_gen(PREFIX, ISO_READ=2,\
                   ISOLIST=['H-1','He-4','C-12','N-14','O-16','Ne-20','Si-28','S-32','Cr-50','Fe-56','Ni-56']):
    import heapq as hq
    import numpy as np
    import matplotlib.pyplot as plt
    #Number of isotopes to examine, 2 most abundant isotopes by default
    num_iso_test=ISO_READ
    mass_coord = PREFIX.se.get(cyc_pre,'mass')
    mc_max_iso,mc_max_x,max_iso,j=[],[],[],0
    iso_dict = {i:PREFIX.se.get(cyc_pre,'iso_massf',i) for i in ISOLIST}
    for mc in mass_coord:
        max_iso_dum=[]
        for i in ISOLIST:
            max_iso_dum.append(iso_dict[i][j])
        iso_name_dict={str(iso_dict[i][j]):i for i in ISOLIST}
        iso_top_two = hq.nlargest(num_iso_test,max_iso_dum)
        mc_max_x.append(iso_top_two)
        mc_max_iso.append([iso_name_dict[str(x)] for x in iso_top_two])
        j = j+1  

    mass_coord_shell_bound,k=[0],0
    for i in mass_coord:
        try:
            if mc_max_iso[k][:1] != mc_max_iso[k+1][:1] or mc_max_iso[k][1:] != mc_max_iso[k+1][1:] :
                if mc_max_iso[k][:1] != mc_max_iso[k+1][1:] and mc_max_iso[k+1][:1] != mc_max_iso[k][1:]:
                    mass_coord_shell_bound.append(k) # Index of change in num_iso_test length elements
        except IndexError:
            print("") #Quick skip of last value - To be cleaned
        k+=1
    mass_coord_shell_bound.append(len(mass_coord)-1) #Add upper limit
    dum1,dum2,ave_x_shell,lim=[],[],[],0
    #Average abundances within ranges
    for i in mass_coord_shell_bound:
        dum1=[]
        dum2=[]
        for j in range(lim,i):
            dum1.append(mc_max_x[j][0])
            dum2.append(mc_max_x[j][1])
        ave_x_shell.append([np.average(dum1),np.average(dum2)])
        lim = i     
    return(mc_max_x,mc_max_iso,mass_coord_shell_bound,ave_x_shell)

## Data generation

In the cells below data is taken from the vos for both the single mass and mass list.
If the vos dataset has no data for the mass input it will locate the closest mass there is data for.

In [ ]:
# MESA
# s_dict=dict_by_masslist(mass_list,ms.star_log(mass=i,Z=z))
# s_dict={x:ms.star_log(mass=x,Z=z) for x in mass_list}

In [ ]:
# SET 1 - ppd_exp is explosive data, ppd_wind is pre explosive
pt_dict_exp={x:mp.se(mass=x,Z=z,type='ppd_exp') for x in mass_list}
pt_dict_pre={x:mp.se(mass=x,Z=z,type='ppd_wind') for x in mass_list}
pt_exp=mp.se(mass=mass,Z=z,type='ppd_exp')
pt_pre=mp.se(mass=mass,Z=z,type='ppd_wind')

# This resets the mass co-ordinate setting for plotting
m_coor_on = False

### Masscut

Here we look for the masscut to be used in plotting. We do not care of what is below the masscut.

This is currently set up for explosive, and are generated as functions for ease of use with mass list options.

In [ ]:
def m_coo_finder(exp_input,pre_input):
    cyc_exp = exp_input.se.cycles[len(exp_input.se.cycles)-1]
    cyc_exp_0 = exp_input.se.cycles[0]

    cyc_pre = pre_input.se.cycles[len(pre_input.se.cycles)-1]

    masscut_output =  max(exp_input.se.get(cyc_exp,'mass'))
    mass_coo = exp_input.se.get(cyc_exp,'mass')
    temp = exp_input.se.get(cyc_exp_0,'temperature')

    mass_coo_min_out = len(temp)
    j=0
    for i in temp:
        if i > 1.05e-9:
            dum = mass_coo[j]
            masscut_output = min(dum,masscut_output)
            mass_coo_min_out = min(j,mass_coo_min_out)
        j=j+1
    print('')
    return(masscut_output,mass_coo_min_out,cyc_exp,cyc_exp_0,cyc_pre)
masscut,mass_coo_min,cyc_exp,cyc_exp_0,cyc_pre=m_coo_finder(pt_dict_exp[mass],pt_dict_pre[mass])

In [ ]:
# Mass list data generation
mass_dict={x:m_coo_finder(pt_dict_exp[x],pt_dict_pre[x]) for x in mass_list}

### Generating temperature variables

This cell will first look for temperature values in a known list. If no values are found it will generate values from the dataset. This is to minimise time wasted on data generation. 

In [ ]:
# exctract Tmax vs mass
def temp_finder(mass,z,mass_coo_min,pt_exp):
    # The file in which the temperature profiles are held for metalicities and masses
    # This is used to allow for quick data generation
    # as creating a new data set can take a while
    try:
        f = open('temp_profile_exp.txt','r')
    except FileNotFoundError:
        f = open('temp_profile_exp.txt','w')
        f.write('%s,%s,%s,%s' %(0,0,0,0))
        f.close()
        f = open('temp_profile_exp.txt','r')
    line = f.readlines()
    fm=[]
    fz=[]
    fcmt=[]
    fmt=[]
    for x in line:
        fm.append(x.split(',')[0])
        fz.append(x.split(',')[1])
        fcmt.append(x.split(',')[2])
        fmt.append(x.split(',')[3])
    #Here we look for the specific mass and Z within the reference file
    for i in range(len(fm)):
        if fm[i] == str(mass) and fz[i] == str(z):
            print('cyc_max_t and max_t found in file')
            cyc_max_t = fcmt[i]
            max_t     = fmt[i][:-2]
            break
            # If profiles found the loop ends with data being output
        elif i == (len(fm)-1): # if no profile found by the end of the file generate a profile
            max_t = 0.
            j = 0
            print('Creating cyc_max_t and max_t')
            print(' ')
            for i in pt_exp.se.cycles:
                try:
                    dum=pt_exp.se.get(i,'temperature')[mass_coo_min]
                    max_t=max(dum,max_t)
                    if dum == max_t:
                        cyc_max_t = i
                except IOError:
                    print(' ')
                    print('Cycle crash on:') # Cycle crashes at various temperatures - CHECK WHY
                    print(j)                 # Currently no obvious effect on results as ~5 out of 1000 bad temps
                j += 1 
            print(' ')
            print('cycles:')
            print(j)
            print(mass)
            print(z)
            write = open('temp_profile_exp.txt','a')
            write.write('\n%s,%s,%s,%s' %(mass,z,cyc_max_t,max_t))
            write.close

        f.close()
    print(cyc_max_t,max_t)
    temperature_peak_array = pt_exp.se.get(cyc_max_t,'temperature')
    print('')
    return (cyc_max_t,max_t,temperature_peak_array)

cyc_max_t,max_t,temperature_peak_array=temp_finder(mass,z,mass_coo_min,pt_dict_exp[mass])
# !cat temp_profile_exp.txt

In [ ]:
# Generation for mass list
temp_dict={x:temp_finder(x,z,mass_dict[x][1],pt_dict_exp[x]) for x in mass_list}

## Plotting abundances

There are a set of boolean parameters that define the plot generated.
* plot_pre       : Include pre explosive abundancies (Dotted plots)
* plot_exp       : Include explosive abundancies     (Solid plots)
* mass_cut_plot  : Include masscut region
* shell_plot     : Include shell structure 
* use_mass_list  : Plots mass list values, defined by mass input
* add_vert_line  : Creates vertical line plots at defined markers, will add names if line names provided. If temp_array is generated will attempt to show peak temperature inputs as mass co-ordinates
* plot_peak_temp : Plots theh peak temperature curve, allowing temp_array to be created by inspection
* save_fig_image : Saves a .png of the plot generated

In [ ]:
# Start by defining the isotopes that need to be plotted
iso = ['He-4','Ti-44','Si-28']
# iso = ['H-1','He-4','C-12','N-14','O-16','Ne-20','Si-28','S-32','Ti-44','Cr-50','Fe-56','Ni-56']
# iso = ['O-16','Si-28','Ni-56']#,'Ne-20','Ni-56','S-32']
lin = '-'
llin='--'

######
plot_pre =         True
plot_exp =         True
mass_cut_plot=     False
shell_plot=        True
# Shell plot generated using two most abundant isotopes
use_mass_list=     False
add_vert_line=     False
plot_peak_temp=    False
save_fig_image=    False
######

mass_input = [12]
# mass_input = mass_list


fig, ax1 = subplots(figsize=(10,5))
ax2 = ax1.twinx()
#ax1.figure(figsize=(9,6))
pre=pt_pre


if shell_plot:
    mass_coord = pre.se.get(cyc_pre,'mass')
    mc_max_x,mc_max_iso,mass_coord_shell_bound,ave_x_shell=shell_data_gen(PREFIX=pre)
    for i in range(len(mass_coord_shell_bound)-1):
        ax2.axvspan(mass_coord[mass_coord_shell_bound[i]],mass_coord[mass_coord_shell_bound[i+1]],facecolor=col[i]\
                    ,alpha=0.25,label='%s/%s' %(mc_max_iso[mass_coord_shell_bound[i+1]-2][0],\
                                                mc_max_iso[mass_coord_shell_bound[i+1]-2][1]))
#     ax1.plot(mass_coord,log10(mc_max_x))

# Core plotting loops for both post and pre explosive data
j=0
if plot_exp:
    if use_mass_list:
        for i in iso:
            for mi in mass_input:
                ax1.plot(pt_dict_exp[mi].se.get(temp_dict[mi][0],'mass'),log10(pt_dict_exp[mi]\
                        .se.get(temp_dict[mi][0],'iso_massf',i)),color=col[j],ls=llin,label='CCSN '+i+','+mi+'M $\odot$')
                j=j+1
    else:
        for i in iso:
            ax1.plot(pt_exp.se.get(cyc_exp,'mass'),log10(pt_exp.se.get(cyc_exp,'iso_massf',i)),color=col[j],ls=lin\
                  ,label='CCSN '+i)
            j=j+1
j=0
if plot_pre:
    if use_mass_list:
        for i in iso:
            for mi in mass_input:
                ax1.plot(pt_dict_pre[mi].se.get(temp_dict[mi][0],'mass'),log10(pt_dict_pre[mi]\
                        .se.get(temp_dict[mi][0],'iso_massf',i)),color=col[j],ls=lin,label='PreSN '+i)
                j=j+1
        
    else:    
        for i in iso:
            ax1.plot(pt_pre.se.get(cyc_pre,'mass'),log10(pt_pre.se.get(cyc_pre,'iso_massf',i)),color=col[j],ls=lin,\
                     label='PreSN: '+i)
            j=j+1

vline=[1.44,1.969]
print("")
prev_val=0.
j=0
if add_vert_line:
    for i in vline:
        plt.axvline(i,color=col[j],lw=0.5)
        if len(vlinename)>1:
            textloc=((i+prev_val)/2)
            plt.text(textloc-0.1,0.2,vlinename[j],color=col[j],fontsize='small')
        prev_val=i
        j += 1
    if len(vlinename)>1:
        plt.text(3.6,0.2,'H',color='k',fontsize='small')
    if m_coor_on:
        [plt.axvline(m_coor[i], color=col[i], lw=1.5,label='Trajectory Extraction') for i in range(len(m_coor))]    

# Add in the mass cut here
if mass_cut_plot:    
    axvline(x=masscut,color='0.25',linewidth=5)  
    axvspan(0.0, masscut, facecolor='0.5', alpha=0.5)
    print('Mass cut: %s' %masscut)


# Plot the peak temperature for reference when generatingtemp_array
if plot_peak_temp:
    ax2.plot(pt_exp.se.get(cyc_max_t,'mass'),pt_exp.se.get(cyc_max_t,'temperature'),'k-+')
    
# PLOTTING PARAMETERS:
ax1.legend(bbox_to_anchor=(1.15, 1), loc=2, borderaxespad=0.)    
if shell_plot:
    ax1.legend(bbox_to_anchor=(1.10, 1), loc=2, borderaxespad=0.)    
    ax2.legend(bbox_to_anchor=(1.3725, 1), loc=2, borderaxespad=0.)    
ax1.set_ylabel('Mass fraction')
ax1.set_ylim(-8,0)
ax1.set_xlim(0.5,4)
ax1.set_xlabel('Mass coordinate [Msun]')
ax2.set_ylabel('Temperature [GK]')

fig.subplots_adjust(right=0.65)
fig.show()

if save_fig_image:
    fig.savefig('abu_temp_peak_plot_M'+str(mass)+'_z'+str(z)+'.png')
    print("IMAGE SAVED:")
    print('abu_temp_peak_plot_M'+str(mass)+'_z'+str(z)+'.png')

## Trajectory Set-up

Above you can see that for all mass coordinates we have peak temperature at the same cycle, cyc_max_t. However, this is not always the case.
Use the plot above in the following way. Check the abundance profiles that you care about, and based on that check the peak temperatures relevant to trigger the production of destruction of them. For example when looking for increases in Ti-44 production a peak can be aligned with the temperature profile. By hovering the mouse above the position on the temperature profile a value is shown in the bottom right of the plot. These temperature are collected below in a list.

In [ ]:
#choose a set of trajectories from a set of peak temperatures
temp_array= [1.478,2.7205,3.7268,4.3459,5.633,6.4558] #Ti44Finaltraj


# Formatting for Traj creation, converting the trajectory position into a mass coordinate.
# This list of mass coordinates are printed 
index_temp_array = []
for i in temp_array:
    index_temp_array.append(numpy.argmin(numpy.abs(i - temperature_peak_array)))
m_coor = []
for i in index_temp_array:
    m_coor.append(pt_exp.se.get(cyc_exp,'mass')[i])
m_coor_on = True
print('')
print(m_coor)

## Trajectory generation

Below is a script that uses the temperature array to generate trajectory.input and iniab files for all vlaues in the list. This portion can take a long time as it requires a large amount of communication with CANFAR. The output is set to create multiple trajectoryX.input and mass_fX.dat files, where each X is the mass coordinate associated with that peak temperature value.

In [ ]:
model_start = int(cyc_exp_0)
model_stop = int(cyc_exp)
r_array = []
rho_array = []
temperature_array = []
time_array = []
j=1
filenames=[]
for i in m_coor:
        r,rho,temperature,time= pt_exp.trajectory(model_start, model_stop, 5,\
                     i, age_in_sec=False, online=False)
        r_array.append(r)
        rho_array.append(rho)
        temperature_array.append(temperature)
        time_array.append(time)
        j = j+1
        print('')
        print('done')
        print('-----------------')
print('All done')

In [ ]:
%rm traj_*dat
%rm trajectory*.input
j=0
for i in m_coor:
    at.writeTraj(filename='trajectory'+str(i)+'.input',\
            data=[time_array[j],temperature_array[j],rho_array[j]],\
            ageunit=1, tunit=1, rhounit=0, idNum=0)
    j=j+1

In [ ]:
# calculate the initial abundance files
%rm massf_*.dat
for i in m_coor:
    pt_exp.abund_at_masscoordinate(model_start,i,online=False)

In [ ]:
pt_exp.se.get('temperature_unit')/1.0e9

# Mass list analysis plots

Here some plots are generated to compare the mass lists properties. The first plot compares the peak temperature and peak density of the supernova shockwave. The second plot shows thereletionship between the peak temperature and abundance, allowing for an examination of temperature and relevent nucleosynthesis. The third plot shows the mass co-ordinate and peak temperature and density, used to identify mass cut regions and as a comparison between mass list showckwave parameters.

In [ ]:
fig, ax1 = subplots(figsize=(9,5))
print('')
# print(min(rho_array))
# ax1.plot(T_array,rho_array,col[0])
T_group=[]
rho_group=[]
j=0
for i in mass_list:
    T_array,T_l = limiter(pt_dict_exp[i].se.get(temp_dict[i][0],'temperature'))
    print(len(T_array))
    rho_array= pt_dict_exp[i].se.get(temp_dict[i][0],'rho')
    rho_array=rho_array[T_l:]
    print(len(rho_array)  )
    ax1.plot(T_array,rho_array,col[j],label='Mass:'+str(i)+' Z='+str(z))
    j=j+1


    
ax1.legend(bbox_to_anchor=(1.15, 1), loc=2, borderaxespad=0.)    
ax1.set_ylabel('Density Peak [g cm$^{-3}$]')
# ax1.set_ylim(-13,0)
ax1.set_xlabel('Temperature Peak [GK]')
# ax1.set_xlim(max(T_array),min(T_array))
# ax1.set_xlim(min(T_array),max(T_array))

fig.subplots_adjust(right=0.65)
fig.show()


In [ ]:
iso = ['He-4','Ti-44','Si-28','Ni-56']
# iso = ['Ti-44','Si-28','Ni-56']
# iso =['Si-28']
style = ['-','--','-.',':']

fig, ax1 = subplots(figsize=(9,5))

T_dict={x:limiter(pt_dict_exp[x].se.get(temp_dict[x][0],'temperature')) for x in mass_list}
# print T_array
iso_array = []
j=0
for i in iso:
    l=0
#     for k in mass_list:
#         dum = log10(pt_dict_exp[k].se.get(mass_dict[k][2],'iso_massf',i))
#         dum = dum[T_dict[k][1]:]
#         ax1.plot(T_dict[k][0],dum,col[j],ls=style[l],label='Mass: '+str(k)+"$M_{\odot}$, Isotope: "+str(i))
#         l=l+1 
    dum = log10(pt_dict_exp[12].se.get(mass_dict[12][2],'iso_massf',i))
    dum = dum[T_dict[12][1]:]
    ax1.plot(T_dict[12][0],dum,col[j],ls=style[0],label='Mass: '+str(12)+"$M_{\odot}$, Isotope: "+str(i)+', PPN')
    dum = log10(pt_dict_exp[15].se.get(mass_dict[15][2],'iso_massf',i))
    dum = dum[T_dict[15][1]:]
    ax1.plot(T_dict[15][0],dum,col[j],ls=style[1],label='Mass: '+str(15)+"$M_{\odot}$, Isotope: "+str(i)+', PPN')
    j=j+1

    
ax1.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)    
ax1.set_ylabel('Mass fraction')
ax1.set_ylim(-8,0)
ax1.set_xlabel('Peak Temperature [GK]')
ax1.set_xlim(7,0)

fig.subplots_adjust(right=0.65)
fig.show()


In [ ]:
# ax1.plot(pt_exp.se.get(cyc_exp,'temperature'),log10(pt_exp.se.get(cyc_exp,'iso_massf',i)),col[j]+llin,label='CCSN '+i)
fig, ax1 = subplots(figsize=(9,5))
ax2=ax1.twinx()
j=0
for i in mass_list:
    temp_array_test=pt_dict_exp[i].se.get(temp_dict[i][0],'temperature')
    rho_array_test=pt_dict_exp[i].se.get(temp_dict[i][0],'rho')
    print('')
    print(temp_array_test[0])
    print(max(temp_array_test))
    xplot=np.linspace(0,len(temp_array_test)-1,num=len(temp_array_test))
    ax1.plot(xplot,temp_array_test,color=col[j],ls='-', label='Temp for: '+str(i)+'M$_{\odot}$')
    ax2.plot(xplot,rho_array_test,color=col[j],ls='-.', label='Density for: '+str(i)+'M$_{\odot}$')
    j+=1
maxt= np.max(temp_array_test)
mint= np.min(temp_array_test)
# ax1.set_ylim(mint,maxt)

ax1.set_ylabel('Temp')
ax2.set_ylabel('Rho')
ax1.set_xlabel('Num')

ax1.legend(bbox_to_anchor=(1.25, 1), loc=2, borderaxespad=0.)    
ax2.legend(bbox_to_anchor=(1.25, 0.65), loc=2, borderaxespad=0.)    

# ax1.plot(xplot2,T_array_20)
fig.subplots_adjust(right=0.65)
fig.show()
